In [1]:
# Importing Packages
import requests 
import json
from time import sleep
import pandas as pd

In [2]:
# Setting up the API key and base URL
API_KEY = "XXXXXXXXXXXXXXXX" #enter in the API code
BASE_URL = f"https://api.themoviedb.org/3"

First, I'm going to gather data on the top 25 movies per year since 1916. (Google said that it was when the first seuqel came out). Right now I only need to get the movie_id.

In [3]:
# Function to get the top movies by year with filtered fields
def get_top_movieID_by_year(year, num_movies=25):
    movies = []
    page = 1

    while len(movies) < num_movies:
        params = {
            'api_key': API_KEY,
            'primary_release_year': year,
            'with_origin_country' : 'US',
            'sort_by': 'revenue.desc',
            'page': page
        }
        response = requests.get(f"https://api.themoviedb.org/3/discover/movie", params=params)
        
        # Check if the request was successful
        if response.status_code != 200:
            print(f"Error {response.status_code} for year {year}, page {page}")
            break
        
        try:
            data = response.json()
        except requests.JSONDecodeError:
            print(f"Failed to decode JSON for year {year}, page {page}")
            break

        if 'results' not in data:
            print(f"No data returned for year {year}, page {page}.")
            break

        for movie in data['results']:
            #retrive the movie id
            movie_id = movie.get("id")
            movies.append(movie_id)
                

        if len(data['results']) < 20:  # Stop if fewer than 20 results on the last page
            break
        page += 1
        #sleep(0.1)  # To avoid hitting the rate limit

    return movies[:num_movies]

In [4]:
# Collect top 25 movies since 1916 (the first year that a sequel was released)
all_movies = []
current_year = 2024 
start_year = 1916

for year in range(start_year, current_year + 1):
    print(f"Fetching data for year {year}...")
    movies = get_top_movieID_by_year(year)
    all_movies.extend(movies)

Fetching data for year 1916...
Fetching data for year 1917...
Fetching data for year 1918...
Fetching data for year 1919...
Fetching data for year 1920...
Fetching data for year 1921...
Fetching data for year 1922...
Fetching data for year 1923...
Fetching data for year 1924...
Fetching data for year 1925...
Fetching data for year 1926...
Fetching data for year 1927...
Fetching data for year 1928...
Fetching data for year 1929...
Fetching data for year 1930...
Fetching data for year 1931...
Fetching data for year 1932...
Fetching data for year 1933...
Fetching data for year 1934...
Fetching data for year 1935...
Fetching data for year 1936...
Fetching data for year 1937...
Fetching data for year 1938...
Fetching data for year 1939...
Fetching data for year 1940...
Fetching data for year 1941...
Fetching data for year 1942...
Fetching data for year 1943...
Fetching data for year 1944...
Fetching data for year 1945...
Fetching data for year 1946...
Fetching data for year 1947...
Fetching

Now that I've got the movie IDs from 1916-2024, I want to gather specific variables for each of them. 

I want each movie's: 
- Title 
- Release Year
- Budget
- Revenue 
- Vote Average
- Collection ID (this will help us determine if the movie is a sequel or not)

In [5]:
# Function to get specific details given a movie id
def get_movie_details(movie_id):
    """Fetches movie details for a given movie ID from TMDb API."""
    url = f'https://api.themoviedb.org/3/movie/{movie_id}'
    params = {
        'api_key': API_KEY,
        'language': 'en-US'
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        # Retrieve only the fields we're interested in
        return {
            'movie_id': movie_id,
            'title': data.get('title'),
            'release_year': data.get('release_date', '').split('-')[0] if data.get('release_date') else None,
            'budget': data.get('budget'),
            'revenue': data.get('revenue'),
            'vote_average': data.get('vote_average'),
            'collection_id': data['belongs_to_collection']['id'] if data.get('belongs_to_collection') else None
        }
    else:
        print(f"Error {response.status_code} for movie ID {movie_id}")
        return None
    
# Funtion to get details from a list of movie id's
def get_movie_data(movie_ids):
    """Fetches movie data for a list of movie IDs and returns it as a DataFrame."""
    movie_data = []

    count = 0
    
    for movie_id in movie_ids:
        details = get_movie_details(movie_id)
        count = count + 1
        if details:
            movie_data.append(details)
        print(f"ID: {movie_id} | {count}/{len(movie_ids)}")

    return pd.DataFrame(movie_data)

In [6]:
# Convert the filtered data to a DataFrame and save to CSV
df = get_movie_data(all_movies)
df.to_csv('top25_per_year.csv', index=False)
print("\nData collection complete. Saved to 'top25_per_year.csv'")

ID: 30266 | 1/2725
ID: 3059 | 2/2725
ID: 278480 | 3/2725
ID: 563515 | 4/2725
ID: 1421151 | 5/2725
ID: 1421146 | 6/2725
ID: 1420341 | 7/2725
ID: 1417839 | 8/2725
ID: 1417835 | 9/2725
ID: 1417832 | 10/2725
ID: 1417830 | 11/2725
ID: 1417342 | 12/2725
ID: 1417336 | 13/2725
ID: 1417331 | 14/2725
ID: 1417328 | 15/2725
ID: 1417325 | 16/2725
ID: 1414916 | 17/2725
ID: 1414914 | 18/2725
ID: 1413487 | 19/2725
ID: 1412496 | 20/2725
ID: 1412491 | 21/2725
ID: 1412488 | 22/2725
ID: 1412481 | 23/2725
ID: 1407166 | 24/2725
ID: 1405405 | 25/2725
ID: 1421719 | 26/2725
ID: 1421713 | 27/2725
ID: 1421665 | 28/2725
ID: 1421664 | 29/2725
ID: 1421660 | 30/2725
ID: 1421658 | 31/2725
ID: 1421656 | 32/2725
ID: 1420682 | 33/2725
ID: 1420675 | 34/2725
ID: 1420052 | 35/2725
ID: 1418364 | 36/2725
ID: 1418363 | 37/2725
ID: 1418362 | 38/2725
ID: 1418360 | 39/2725
ID: 1418359 | 40/2725
ID: 1418355 | 41/2725
ID: 1417843 | 42/2725
ID: 1416854 | 43/2725
ID: 1416848 | 44/2725
ID: 1416845 | 45/2725
ID: 1416843 | 46/2725
ID: 

ID: 1398968 | 748/2725
ID: 1385235 | 749/2725
ID: 1383604 | 750/2725
ID: 240417 | 751/2725
ID: 13850 | 752/2725
ID: 303 | 753/2725
ID: 887 | 754/2725
ID: 32275 | 755/2725
ID: 910 | 756/2725
ID: 1585 | 757/2725
ID: 18700 | 758/2725
ID: 3767 | 759/2725
ID: 25736 | 760/2725
ID: 44098 | 761/2725
ID: 44100 | 762/2725
ID: 27033 | 763/2725
ID: 20246 | 764/2725
ID: 26286 | 765/2725
ID: 32945 | 766/2725
ID: 27452 | 767/2725
ID: 16090 | 768/2725
ID: 3088 | 769/2725
ID: 263013 | 770/2725
ID: 1841 | 771/2725
ID: 43483 | 772/2725
ID: 25502 | 773/2725
ID: 44042 | 774/2725
ID: 1411003 | 775/2725
ID: 33667 | 776/2725
ID: 121357 | 777/2725
ID: 27437 | 778/2725
ID: 108688 | 779/2725
ID: 87531 | 780/2725
ID: 16227 | 781/2725
ID: 175386 | 782/2725
ID: 217155 | 783/2725
ID: 11881 | 784/2725
ID: 62720 | 785/2725
ID: 33478 | 786/2725
ID: 452261 | 787/2725
ID: 14595 | 788/2725
ID: 31667 | 789/2725
ID: 19335 | 790/2725
ID: 192501 | 791/2725
ID: 3766 | 792/2725
ID: 119102 | 793/2725
ID: 28120 | 794/2725
ID: 469

ID: 40894 | 1130/2725
ID: 36362 | 1131/2725
ID: 22386 | 1132/2725
ID: 821 | 1133/2725
ID: 164 | 1134/2725
ID: 11536 | 1135/2725
ID: 990 | 1136/2725
ID: 2160 | 1137/2725
ID: 24248 | 1138/2725
ID: 248 | 1139/2725
ID: 18647 | 1140/2725
ID: 28284 | 1141/2725
ID: 18228 | 1142/2725
ID: 430 | 1143/2725
ID: 30462 | 1144/2725
ID: 20139 | 1145/2725
ID: 28501 | 1146/2725
ID: 98010 | 1147/2725
ID: 84119 | 1148/2725
ID: 16372 | 1149/2725
ID: 84249 | 1150/2725
ID: 11349 | 1151/2725
ID: 9289 | 1152/2725
ID: 11897 | 1153/2725
ID: 11085 | 1154/2725
ID: 595 | 1155/2725
ID: 39391 | 1156/2725
ID: 802 | 1157/2725
ID: 13671 | 1158/2725
ID: 11697 | 1159/2725
ID: 982 | 1160/2725
ID: 31804 | 1161/2725
ID: 43012 | 1162/2725
ID: 41521 | 1163/2725
ID: 898 | 1164/2725
ID: 76344 | 1165/2725
ID: 62655 | 1166/2725
ID: 1162 | 1167/2725
ID: 42995 | 1168/2725
ID: 29235 | 1169/2725
ID: 124138 | 1170/2725
ID: 202875 | 1171/2725
ID: 31287 | 1172/2725
ID: 1421846 | 1173/2725
ID: 1419524 | 1174/2725
ID: 1415536 | 1175/2725
I

ID: 11558 | 1508/2725
ID: 10649 | 1509/2725
ID: 23479 | 1510/2725
ID: 10970 | 1511/2725
ID: 12268 | 1512/2725
ID: 7340 | 1513/2725
ID: 6037 | 1514/2725
ID: 10747 | 1515/2725
ID: 103 | 1516/2725
ID: 42239 | 1517/2725
ID: 16084 | 1518/2725
ID: 10803 | 1519/2725
ID: 5228 | 1520/2725
ID: 10774 | 1521/2725
ID: 10518 | 1522/2725
ID: 32037 | 1523/2725
ID: 26842 | 1524/2725
ID: 42252 | 1525/2725
ID: 11 | 1526/2725
ID: 840 | 1527/2725
ID: 11009 | 1528/2725
ID: 11319 | 1529/2725
ID: 11006 | 1530/2725
ID: 14741 | 1531/2725
ID: 20416 | 1532/2725
ID: 5902 | 1533/2725
ID: 24032 | 1534/2725
ID: 703 | 1535/2725
ID: 83061 | 1536/2725
ID: 12535 | 1537/2725
ID: 11586 | 1538/2725
ID: 7227 | 1539/2725
ID: 14140 | 1540/2725
ID: 11590 | 1541/2725
ID: 17689 | 1542/2725
ID: 12262 | 1543/2725
ID: 37749 | 1544/2725
ID: 42222 | 1545/2725
ID: 19050 | 1546/2725
ID: 30315 | 1547/2725
ID: 12637 | 1548/2725
ID: 31037 | 1549/2725
ID: 11940 | 1550/2725
ID: 621 | 1551/2725
ID: 1924 | 1552/2725
ID: 579 | 1553/2725
ID: 846

ID: 9079 | 1893/2725
ID: 1497 | 1894/2725
ID: 10333 | 1895/2725
ID: 11384 | 1896/2725
ID: 10276 | 1897/2725
ID: 9319 | 1898/2725
ID: 4032 | 1899/2725
ID: 650 | 1900/2725
ID: 812 | 1901/2725
ID: 619 | 1902/2725
ID: 772 | 1903/2725
ID: 402 | 1904/2725
ID: 943 | 1905/2725
ID: 364 | 1906/2725
ID: 881 | 1907/2725
ID: 2005 | 1908/2725
ID: 6114 | 1909/2725
ID: 8872 | 1910/2725
ID: 9869 | 1911/2725
ID: 8077 | 1912/2725
ID: 33 | 1913/2725
ID: 8845 | 1914/2725
ID: 9374 | 1915/2725
ID: 11806 | 1916/2725
ID: 11259 | 1917/2725
ID: 9475 | 1918/2725
ID: 11287 | 1919/2725
ID: 11066 | 1920/2725
ID: 10326 | 1921/2725
ID: 2322 | 1922/2725
ID: 9349 | 1923/2725
ID: 10407 | 1924/2725
ID: 10158 | 1925/2725
ID: 329 | 1926/2725
ID: 788 | 1927/2725
ID: 424 | 1928/2725
ID: 37233 | 1929/2725
ID: 4478 | 1930/2725
ID: 9350 | 1931/2725
ID: 858 | 1932/2725
ID: 9800 | 1933/2725
ID: 9944 | 1934/2725
ID: 5503 | 1935/2725
ID: 9386 | 1936/2725
ID: 864 | 1937/2725
ID: 1634 | 1938/2725
ID: 1049 | 1939/2725
ID: 9593 | 1940/2

ID: 2268 | 2287/2725
ID: 6477 | 2288/2725
ID: 4523 | 2289/2725
ID: 298 | 2290/2725
ID: 1979 | 2291/2725
ID: 5559 | 2292/2725
ID: 4982 | 2293/2725
ID: 5174 | 2294/2725
ID: 11199 | 2295/2725
ID: 7326 | 2296/2725
ID: 1250 | 2297/2725
ID: 4964 | 2298/2725
ID: 2310 | 2299/2725
ID: 23827 | 2300/2725
ID: 155 | 2301/2725
ID: 217 | 2302/2725
ID: 9502 | 2303/2725
ID: 8960 | 2304/2725
ID: 11631 | 2305/2725
ID: 10527 | 2306/2725
ID: 1726 | 2307/2725
ID: 10681 | 2308/2725
ID: 2454 | 2309/2725
ID: 4564 | 2310/2725
ID: 1735 | 2311/2725
ID: 8966 | 2312/2725
ID: 8909 | 2313/2725
ID: 4922 | 2314/2725
ID: 13053 | 2315/2725
ID: 12222 | 2316/2725
ID: 13223 | 2317/2725
ID: 7840 | 2318/2725
ID: 1724 | 2319/2725
ID: 14306 | 2320/2725
ID: 11887 | 2321/2725
ID: 10200 | 2322/2725
ID: 11665 | 2323/2725
ID: 10201 | 2324/2725
ID: 8247 | 2325/2725
ID: 19995 | 2326/2725
ID: 8355 | 2327/2725
ID: 8373 | 2328/2725
ID: 14161 | 2329/2725
ID: 14160 | 2330/2725
ID: 18239 | 2331/2725
ID: 10528 | 2332/2725
ID: 13448 | 2333/27

ID: 453395 | 2654/2725
ID: 438148 | 2655/2725
ID: 505642 | 2656/2725
ID: 414906 | 2657/2725
ID: 616037 | 2658/2725
ID: 315162 | 2659/2725
ID: 338953 | 2660/2725
ID: 335787 | 2661/2725
ID: 675353 | 2662/2725
ID: 436270 | 2663/2725
ID: 614934 | 2664/2725
ID: 629542 | 2665/2725
ID: 718930 | 2666/2725
ID: 718789 | 2667/2725
ID: 882598 | 2668/2725
ID: 539681 | 2669/2725
ID: 752623 | 2670/2725
ID: 536554 | 2671/2725
ID: 762504 | 2672/2725
ID: 800939 | 2673/2725
ID: 526896 | 2674/2725
ID: 756999 | 2675/2725
ID: 346698 | 2676/2725
ID: 502356 | 2677/2725
ID: 872585 | 2678/2725
ID: 447365 | 2679/2725
ID: 385687 | 2680/2725
ID: 569094 | 2681/2725
ID: 447277 | 2682/2725
ID: 575264 | 2683/2725
ID: 976573 | 2684/2725
ID: 640146 | 2685/2725
ID: 667538 | 2686/2725
ID: 603692 | 2687/2725
ID: 572802 | 2688/2725
ID: 615656 | 2689/2725
ID: 335977 | 2690/2725
ID: 695721 | 2691/2725
ID: 940551 | 2692/2725
ID: 507089 | 2693/2725
ID: 677179 | 2694/2725
ID: 298618 | 2695/2725
ID: 968051 | 2696/2725
ID: 1160164

That initial dataset is done!

Now onto getting collection info. Collections are basically how TMDB groups franchise movies together. 
This will help us add a new variable "is_sequel" later on (in the R code).

First, I need to gather all of the collection IDs for my top 25 movies

In [7]:
# Load the CSV file (replace 'your_file.csv' with the actual file path)
df = pd.read_csv('top25_per_year.csv')

# Extract a column into a list (replace 'column_name' with the actual column name)
collections = df['collection_id'].tolist()

# Display the list
print(collections)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [8]:
collec_clean = [x for x in collections if pd.notna(x)]
collec_clean = list(set(collec_clean))
collec_clean = [int(x) for x in collec_clean]
collec_clean

[10,
 86027,
 86028,
 86029,
 1157137,
 722961,
 518165,
 124950,
 124949,
 722971,
 618529,
 1062948,
 86055,
 14377,
 86058,
 43055,
 86066,
 507956,
 43064,
 43066,
 92222,
 43072,
 86083,
 84,
 145494,
 434270,
 114783,
 86115,
 813157,
 2150,
 86119,
 119,
 55419,
 221308,
 108669,
 55422,
 55427,
 397444,
 264338,
 387219,
 108693,
 4246,
 151,
 735384,
 401562,
 702624,
 8354,
 393379,
 327871,
 157890,
 59586,
 90306,
 1231053,
 1036499,
 10453,
 721111,
 10456,
 96472,
 295130,
 131292,
 985311,
 131296,
 131295,
 809185,
 14563,
 230,
 1370345,
 366826,
 209131,
 90348,
 162028,
 1097972,
 168182,
 135416,
 489724,
 573693,
 948485,
 263,
 264,
 956687,
 106768,
 135441,
 37139,
 1149208,
 215326,
 272673,
 86307,
 295,
 2344,
 256297,
 598314,
 86311,
 983340,
 135468,
 33071,
 304,
 422193,
 51509,
 135483,
 33085,
 2366,
 248126,
 86336,
 135495,
 328,
 135498,
 86347,
 123213,
 694606,
 135508,
 100693,
 4438,
 751961,
 8537,
 1415515,
 221537,
 221539,
 104833,
 8580,
 8

Now that I have the collection ID's, I'm going to gather ALL of the movies from those collections. 

There may have been a more efficient way to do this, but I plan on sorting all of the movies in each collection by year.
This allows me to determine the order of every movie in each franchise (i.e. 1st, 2nd, 3rd, etc.). Once I have the franchise order of each movie, I'll merge it with my top25 dataset to add a variable for the movie's order.

In [11]:
## Defining functions

def get_movies_in_collection(collection_id):
    """Fetches movies in a specific collection by collection ID."""
    url = f'https://api.themoviedb.org/3/collection/{collection_id}'
    params = {
        'api_key': API_KEY
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        # Extract movies in the collection with movie_id and release year
        movies = [
            {
                'collection_id': collection_id,
                'movie_id': movie.get('id'),
                'release_year': movie.get('release_date', '').split('-')[0]
            }
            for movie in data.get('parts', [])
            if movie.get('release_date')  # Only include movies with a release date
        ]
        return movies
    else:
        print(f"Error {response.status_code} for collection ID {collection_id}")
        return []

def get_all_movies_in_collections(collection_ids):
    """Fetches all movies in a list of collection IDs and returns them as a DataFrame."""
    all_movies = []
    
    count = 0

    for collection_id in collection_ids:
        count = count+1
        print(f"Fetching movies for collection ID {collection_id}... |{count}/{len(collection_ids)} ")
        movies = get_movies_in_collection(collection_id)
        all_movies.extend(movies)
        sleep(0.1)  # To avoid hitting the rate limit

    return pd.DataFrame(all_movies)


In [12]:
# Get all movies in the specified collections and save to CSV
df = get_all_movies_in_collections(collec_clean)
print(df)
df.to_csv('movies_in_specified_collections.csv', index=False)
print("Data collection complete. Saved to 'movies_in_specified_collections.csv'")

Fetching movies for collection ID 10... |1/444 
Fetching movies for collection ID 86027... |2/444 
Fetching movies for collection ID 86028... |3/444 
Fetching movies for collection ID 86029... |4/444 
Fetching movies for collection ID 1157137... |5/444 
Fetching movies for collection ID 722961... |6/444 
Fetching movies for collection ID 518165... |7/444 
Fetching movies for collection ID 124950... |8/444 
Fetching movies for collection ID 124949... |9/444 
Fetching movies for collection ID 722971... |10/444 
Fetching movies for collection ID 618529... |11/444 
Fetching movies for collection ID 1062948... |12/444 
Fetching movies for collection ID 86055... |13/444 
Fetching movies for collection ID 14377... |14/444 
Fetching movies for collection ID 86058... |15/444 
Fetching movies for collection ID 43055... |16/444 
Fetching movies for collection ID 86066... |17/444 
Fetching movies for collection ID 507956... |18/444 
Fetching movies for collection ID 43064... |19/444 
Fetching movi

Fetching movies for collection ID 8783... |157/444 
Fetching movies for collection ID 359005... |158/444 
Fetching movies for collection ID 100965... |159/444 
Fetching movies for collection ID 96871... |160/444 
Fetching movies for collection ID 217704... |161/444 
Fetching movies for collection ID 100970... |162/444 
Fetching movies for collection ID 930413... |163/444 
Fetching movies for collection ID 57971... |164/444 
Fetching movies for collection ID 8819... |165/444 
Fetching movies for collection ID 180854... |166/444 
Fetching movies for collection ID 584320... |167/444 
Fetching movies for collection ID 125570... |168/444 
Fetching movies for collection ID 645... |169/444 
Fetching movies for collection ID 125574... |170/444 
Fetching movies for collection ID 389767... |171/444 
Fetching movies for collection ID 150156... |172/444 
Fetching movies for collection ID 762512... |173/444 
Fetching movies for collection ID 88735... |174/444 
Fetching movies for collection ID 1091

Fetching movies for collection ID 623911... |311/444 
Fetching movies for collection ID 9521... |312/444 
Fetching movies for collection ID 271668... |313/444 
Fetching movies for collection ID 101688... |314/444 
Fetching movies for collection ID 431417... |315/444 
Fetching movies for collection ID 87359... |316/444 
Fetching movies for collection ID 945475... |317/444 
Fetching movies for collection ID 259401... |318/444 
Fetching movies for collection ID 216395... |319/444 
Fetching movies for collection ID 605516... |320/444 
Fetching movies for collection ID 386382... |321/444 
Fetching movies for collection ID 531799... |322/444 
Fetching movies for collection ID 939352... |323/444 
Fetching movies for collection ID 404825... |324/444 
Fetching movies for collection ID 492896... |325/444 
Fetching movies for collection ID 453993... |326/444 
Fetching movies for collection ID 968052... |327/444 
Fetching movies for collection ID 1285493... |328/444 
Fetching movies for collection